In [50]:
import uproot
import ROOT
import numpy as np
import pandas as pd
from collections import OrderedDict

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('default')
plt.rcParams['grid.linestyle'] = ':'

In [51]:
import sys
sys.version_info

sys.version_info(major=3, minor=6, micro=3, releaselevel='final', serial=0)

In [58]:
masses = [('60','20'),('52p5','5p0'),('6p0','2p0'),('5p25','0p5')]
ctaus = [1, 10, 100, 1000]
# triggerPath = 'TRIG_dsa_HLT_DoubleMu3_DCA_PFMET50_PFMHT60'
triggerPath = 'TRIGself_dsa_HLT_PFMET130_PFMHT130'
baseDir = '../Firefighter/washAOD/trigger2017Studies/'
fileName = lambda Mchi, dMchi, ctau: baseDir + f'Mchi-{Mchi}_dMchi-{dMchi}_ctau-{ctau}.root'

numParams = len(masses)*len(ctaus)
countMasses = 0

dfs = OrderedDict({})
numEvents = OrderedDict({})
for (Mchi, dMchi) in masses:
    countCtaus = 1
    dfs[Mchi] = OrderedDict({})
    numEvents[Mchi] = OrderedDict({})
    for ctau in ctaus:
        dfs[Mchi][ctau] = uproot.open(fileName(Mchi, dMchi, ctau))[triggerPath + '/trigSelfEffiForMuTrack'].pandas.df()
#         numEvents[Mchi][ctau] = uproot.open(fileName(Mchi, dMchi, ctau))[triggerPath + '/overallInfo'].pandas.df()
        print(f'{countMasses*len(masses)+countCtaus} of {numParams}: ' + fileName(Mchi, dMchi, ctau))
        countCtaus += 1
    countMasses += 1

1 of 16: ../Firefighter/washAOD/trigger2017Studies/Mchi-60_dMchi-20_ctau-1.root
2 of 16: ../Firefighter/washAOD/trigger2017Studies/Mchi-60_dMchi-20_ctau-10.root
3 of 16: ../Firefighter/washAOD/trigger2017Studies/Mchi-60_dMchi-20_ctau-100.root
4 of 16: ../Firefighter/washAOD/trigger2017Studies/Mchi-60_dMchi-20_ctau-1000.root
5 of 16: ../Firefighter/washAOD/trigger2017Studies/Mchi-52p5_dMchi-5p0_ctau-1.root
6 of 16: ../Firefighter/washAOD/trigger2017Studies/Mchi-52p5_dMchi-5p0_ctau-10.root
7 of 16: ../Firefighter/washAOD/trigger2017Studies/Mchi-52p5_dMchi-5p0_ctau-100.root
8 of 16: ../Firefighter/washAOD/trigger2017Studies/Mchi-52p5_dMchi-5p0_ctau-1000.root
9 of 16: ../Firefighter/washAOD/trigger2017Studies/Mchi-6p0_dMchi-2p0_ctau-1.root
10 of 16: ../Firefighter/washAOD/trigger2017Studies/Mchi-6p0_dMchi-2p0_ctau-10.root
11 of 16: ../Firefighter/washAOD/trigger2017Studies/Mchi-6p0_dMchi-2p0_ctau-100.root
12 of 16: ../Firefighter/washAOD/trigger2017Studies/Mchi-6p0_dMchi-2p0_ctau-1000.root

In [18]:
# Non-self trigger analyzer (TRIG_dsa_stuff)

for (Mchi, _dfs) in dfs.items():
    _dict = {}
    _numEvents = {}
    for ctau, df in _dfs.items():
        _counts = df['fired'].value_counts()
        #_counts /= _counts.sum()
        _dict[ctau] = _counts
        _numEvents[ctau] = len(numEvents[Mchi][ctau])

        evDf = pd.DataFrame(_dict).rename(index={0:'Initial selection', 1:'Triggered'})
        evDf = evDf.append(pd.DataFrame([_numEvents]).rename(index={0:'All events'}))
        evDf.loc['Triggered/All (%)'] = 100*evDf.iloc[1] / evDf.iloc[2]
    print(Mchi)
    display(evDf.round(2))

60


,1,10,100,1000
Initial selection,25570.00,26864.0,26172.00,22965.00
Triggered,2440.00,2592.0,3012.00,2925.00
All events,272990.00,286624.0,285737.00,285573.00
Triggered/All (%),0.89,0.9,1.05,1.02


52p5


,1,10,100,1000
Initial selection,1314.00,1268.00,1245.00,1106.00
Triggered,669.00,677.00,738.00,508.00
All events,194340.00,193867.00,194222.00,193286.00
Triggered/All (%),0.34,0.35,0.38,0.26


6p0


,1,10,100,1000
Initial selection,1778.00,1643.00,1544.00,671.00
Triggered,86.00,92.00,92.00,23.00
All events,190396.00,189631.00,190203.00,190990.00
Triggered/All (%),0.05,0.05,0.05,0.01


5p25


,1,10,100,1000
Initial selection,34.00,33.00,26.00,17.0
Triggered,38.00,30.00,16.00,3.0
All events,235156.00,259773.00,247514.00,246612.0
Triggered/All (%),0.02,0.01,0.01,0.0


In [59]:
# Self-trigger analyzer (TRIGself_dsa_stuff)

for (Mchi, _dfs) in dfs.items():
    _dict = {}
    for ctau, df in _dfs.items():
        _counts = df['fired'].value_counts()
        total = _counts.sum()
#         _counts /= _counts.sum()
#         _counts *= 100
        _counts.loc['Total'] = total
        _counts.loc['Fraction fired (%)'] = _counts.iloc[1] / total * 100
        _dict[ctau] = _counts

    evDf = pd.DataFrame(_dict)
    #.rename(index={0:'Initial selection', 1:'Triggered'})
        #evDf = evDf.append(pd.DataFrame([_numEvents]).rename(index={0:'All events'}))
        #evDf.loc['Triggered/All (%)'] = 100*evDf.iloc[1] / evDf.iloc[2]
    print(Mchi)
    display(evDf.round(3))

60


,1,10,100,1000
False,262925.000,275686.000,273998.000,272351.00
True,10065.000,10938.000,11739.000,13222.00
Total,272990.000,286624.000,285737.000,285573.00
Fraction fired (%),3.687,3.816,4.108,4.63


52p5


,1,10,100,1000
False,184939.000,184326.000,184511.0,183358.000
True,9401.000,9541.000,9711.0,9928.000
Total,194340.000,193867.000,194222.0,193286.000
Fraction fired (%),4.837,4.921,5.0,5.136


6p0


,1,10,100,1000
False,189969.000,189171.000,189651.00,190348.000
True,427.000,460.000,552.00,642.000
Total,190396.000,189631.000,190203.00,190990.000
Fraction fired (%),0.224,0.243,0.29,0.336


5p25


,1,10,100,1000
False,234451.0,259055.000,246760.000,245817.000
True,705.0,718.000,754.000,795.000
Total,235156.0,259773.000,247514.000,246612.000
Fraction fired (%),0.3,0.276,0.305,0.322
